## FUNCION 

Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que debería ser el target de un hipotético modelo de regresión, es decir debe ser una variable numérica continua o discreta pero con alta cardinalidad y una variable float "pvalue" cuyo valor por defecto será 0.05.

La función debe devolver una lista con las columnas categóricas del dataframe cuyo test de relación con la columna designada por 'target_col' supere en confianza estadística el test de relación que sea necesario hacer (es decir la función debe poder escoger cuál de los dos test que hemos aprendido tiene que hacer).

La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable numérica continua del dataframe.

#### pasos a seguir
Argumentos: 
    df, target_col, pvalue = 00.5
devolver:
    lista de columnas categóricas cuyo test de relación con la columna targer supera la confianza estadistica del teste de relafcion 


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency



In [2]:

def test_relacion_categoricas(df, target_col, pvalue=0.05):
    # Comprobar si 'target_col' es una columna numérica continua
    if target_col not in df.columns or not pd.api.types.is_numeric_dtype(df[target_col]): #pd.api.types.is_numeric_dtype función está diseñada para verificar si un tipo de datos dado es numérico.
        print(f"Error: '{target_col}' no es una columna numérica continua válida.")
        return None
    
    # Filtrar columnas categóricas
    cols_categoricas = df.select_dtypes(include=["object", "category"]).columns
    
    # Comprobar si hay columnas categóricas
    if len(cols_categoricas) == 0:
        print("Error: No hay columnas categóricas en el dataframe.")
        return None
    
    # Calcular el número de categorías distintas en cada columna categórica
    num_categorias = df[cols_categoricas].nunique()
    
    # Seleccionar la columna categórica con mayor cardinalidad
    col_max_cardinalidad = num_categorias.idxmax()
    
    # Seleccionar el test de relación adecuado
    if pd.api.types.is_numeric_dtype(df[col_max_cardinalidad]):
        test_result = ttest_ind(df[col_max_cardinalidad], df[target_col])
    else:
        contingency_table = pd.crosstab(df[col_max_cardinalidad], df[target_col])
        test_result = chi2_contingency(contingency_table)
    
    # Evaluar el p-value del test de relación
    if test_result.pvalue < pvalue:
        return [col_max_cardinalidad]
    else:
        print(f"No se encontraron relaciones estadísticamente significativas para '{target_col}'.")
        return []




In [3]:

def test_relacion_categoricas(df, target_col, pvalue=0.05):
    """
    Realiza pruebas estadísticas para evaluar la relación entre una columna numérica continua
    (target_col) y columnas categóricas en un DataFrame.

    Parámetros:
    - df: DataFrame, el conjunto de datos.
    - target_col: str, nombre de la columna objetivo (debe ser numérica continua).
    - pvalue: float, umbral de valor p para determinar la significancia estadística.

    Retorna:
    - Lista de nombres de columnas categóricas con relaciones estadísticamente significativas.

    Si hay errores o no se encuentran relaciones significativas, se imprime un mensaje y retorna None.
    """
    lista = []      # Lista para almacenar resultados intermedios
    lista2 = []     # Lista para almacenar columnas con relaciones estadísticamente significativas

    # Comprobar si 'target_col' es una columna numérica continua
    if target_col not in df.columns or not pd.api.types.is_numeric_dtype(df[target_col]): 
        '''Verifica dos condiciones: Si target_col no está presente en las columnas del 
    DataFrame (df.columns) y Si el tipo de datos de la columna target_col no es numérico continuo'''
        print(f"Error: '{target_col}' no es una columna numérica continua válida.")
        return None
    
    # Filtrar columnas categóricas
    col_max_cardinalidad = df.select_dtypes(include=["object", "category"]).columns #especifica que solo se deben incluir las columnas con tipos de datos 'object' (cadenas de texto) o 'category' (columnas categóricas).
    
    # Comprobar si hay columnas categóricas
    if len(col_max_cardinalidad) == 0:
        print("Error: No hay columnas categóricas en el dataframe.")
        return None
    
    # Iterar sobre las columnas categóricas con mayor cardinalidad
    for col in col_max_cardinalidad:
        # Verificar si la columna es numérica
        if pd.api.types.is_numeric_dtype(df[col]): #Si la columna es numérica, realiza la prueba t de independencia (ttest_ind)
            # Realizar la prueba t de independencia
            test_result = ttest_ind(df[col], df[target_col]) #La prueba t evalúa si hay diferencias significativas entre las medias de dos grupos (en este caso, las muestras de la columna categórica y la columna objetivo)
            lista.append((col, test_result))  # Agregar el resultado a la lista
        else: #Si la columna no es numérica, significa que es categórica.
            # Crear una tabla de contingencia y realizar el test de chi-cuadrado
            contingency_table = pd.crosstab(df[col], df[target_col])#crea una tabla de contingencia, tabla que muestra la distribución conjunta de las frecuencias de dos o más variables categóricas.
            test_result = chi2_contingency(contingency_table) #El test de chi-cuadrado evalúa si existe una asociación significativa entre las variables categóricas.
            lista.append((col, test_result))  # Agregar el resultado a la lista
    
    # Evaluar el p-value del test de relación
    for col, result in lista:
        if result.pvalue < pvalue:
            lista2.append(col)  # Agregar la columna con relación significativa a la lista2
    
    # Imprimir mensaje si no se encontraron relaciones significativas
    if not lista2:
        print(f"No se encontraron relaciones estadísticamente significativas para '{target_col}'.")
    
    # Devolver la lista de columnas con relaciones significativas
    return lista2


In [4]:
df = pd.read_csv("./data/hard_to_find/obligatoria_hard.csv",sep = "|")
target = "MEDV"

In [5]:
test_relacion_categoricas(df, "MEDV", pvalue=0.05)

Error: No hay columnas categóricas en el dataframe.


In [6]:
df_inmo = pd.read_csv("./data_2/ejemplo_housing.csv")

In [7]:

columnas_relacionadas = test_relacion_categoricas(df_inmo, 'median_house_value')
print(columnas_relacionadas)

['ocean_proximity']
